In [11]:
from google.cloud import bigquery
import pandas as pd

# Utwórz klienta BigQuery
client = bigquery.Client(project='cybersyn-417809')

# Zdefiniuj zapytanie SQL
sql = """
SELECT distinct store_name FROM `cybersyn-417809.msoszko_dev.stg_dim_stores`
"""

# Wykonaj zapytanie
query_job = client.query(sql)

# Pobierz wyniki zapytania do DataFrame
df = pd.DataFrame([row.values() for row in query_job.result()], columns=["store_name"])

# Wyświetl dane
df.to_csv('store_names_list.csv', index=False)
df

c:\Users\Michał Soszko\AppData\Local\Programs\Python\Python38-32\lib\site-packages\google\auth\_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


,store_name
0,HILLSTREET NEWS AND TOBACCO
1,JAMBOREE FOODS
2,SYCAMORE CONVENIENCE
3,DOUBLE D LIQUOR STORE
4,"CENTRAL CITY LIQUOR, INC."
...,...
2173,SWELL LIQUOR
2174,THOME ENTERPRISES LLC
2175,LONELY OAK DISTILERY
2176,"CATS EYE DISTILLERY, LLC"


In [7]:
import openai
import json

# API Key
#openai.api_key = 'XXXXXX'

# Stores list
shop_names = df['store_name'].tolist()

# hardcoded split for shop names (enforced by openai api for performance reasons)
shop_names_list = [
    shop_names[0:100],
    shop_names[100:200],
    shop_names[200:300],
    shop_names[300:400],
    shop_names[400:500],
    shop_names[500:600],
    shop_names[600:700],
    shop_names[700:800],
    shop_names[800:900],
    shop_names[900:1000],
    shop_names[1000:1100],
    shop_names[1100:1200],
    shop_names[1200:1300],
    shop_names[1300:1400],
    shop_names[1400:1500],
    shop_names[1500:1600],
    shop_names[1600:1700],
    shop_names[1700:1800],
    shop_names[1800:1900],
    shop_names[1900:2000],
    shop_names[2000:2100],
    shop_names[2100:2200],
    shop_names[2200:2300]
]

# Variable for storing company names (brands)
brand_names_list = []

# Call OpenAI to process the list in search and categorize of possible company names (in PL)
for idx, shop_names in enumerate(shop_names_list):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", 
            "content": f"""Dla listy nazw sklepów: {shop_names} Wydedukuj nazwę marek tych sklepów. Sklepów jest dużo a nazwa marki jest zawsze częścią nazwy sklepu. Wynik zwróc w formacie JSON, gdzie mamy słownik: store_name: brand_name. DŁugość listy musi być zawsze identyczna jak liczba podanych nazw sklepów """}
        ]
    )

    # read the responseand store in brand_names var
    brand_names = response.choices[0].message.content
    # update list  of brands
    brand_names_list.append(brand_names)

In [10]:
# Convert GPT output to a list of strings
df_to_csv = pd.DataFrame(brand_names_list)

# Save the list as csv for further cleaning
df_to_csv.to_csv('raw_list_from_GPT.csv', index=False)
